# Delta Lake Time Machine & Optimization Lab

Delta Lake allows you to read, write and query data in data lakes in an efficient manner.

## In this lesson you:
* Compare different versions of a Delta table using Time Machine
* Optimize your Delta Lake to increase speed and reduce number of files

## Audience
* Primary Audience: Data Engineers
* Secondary Audience: Data Analysts and Data Scientists

## Prerequisites
* Web browser: current versions of Google Chrome, Firefox, Safari, Microsoft Edge and
Internet Explorer 11 on Windows 7, 8, or 10 (see <a href="https://docs.databricks.com/user-guide/supported-browsers.html#supported-browsers#" target="_blank">Supported Web Browsers</a>)
* Databricks Runtime 4.2 or greater
## Datasets Used
We will use online retail datasets from `/mnt/training/online_retail`

In [0]:
%run "./Includes/Classroom-Setup"

Initialized classroom variables & functions...

Datasets are already mounted to /mnt/training from s3a://databricks-corp-training/common

Created user-specific database

Using the database vanitha_a_sivakumar_gmail_com_db .

All done!

Because we'll be calculating some aggregates in this notebook, we'll change our partitions after shuffle from the default `200` to `8` (which is a good number for the 8 node cluster we're currently working on).

In [0]:
%python

sqlContext.setConf("spark.sql.shuffle.partitions", "8")

### Check for our previous Delta Lake tables

This lab relies upon some tables created in previous Delta Lake lessons and labs. 

If you get an error from either of the next two SQL queries, running the solution code for the "Delta-Lake-Lab-1" will build all necessary tables.

In [0]:
%sql

SELECT COUNT(*) as customer_counts FROM customer_counts;
SELECT COUNT(*) as customer_data_delta FROM customer_data_delta;

customer_data_delta
165544


-sandbox
<img alt="Caution" title="Caution" style="vertical-align: text-bottom; position: relative; height:1.3em; top:0.0em" src="https://files.training.databricks.com/static/images/icon-warning.svg"/> **The following cell will take several minutes to execute, and is only necessary to run if you got an error in the previous cell.**

In [0]:
%run "./Includes/Delta-Lab-2-Prep"

Initialized classroom variables & functions...

Datasets are already mounted to /mnt/training from s3a://databricks-corp-training/common

Created user-specific database

Using the database vanitha_a_sivakumar_gmail_com_db .

All done!

Out[13]: False

Out[15]: DataFrame[]

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
14,4,0,10


Out[20]: DataFrame[]

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
38138,14,0,38124


For convenience later in this lab, the paths to the files defining our existing Delta tables are provided. You can use these paths to load the data into DataFrames, if desired, though this entire lab can be completed using SQL on the existant tables.

In [0]:
DeltaPath = userhome + "/delta/customer-data/"
CustomerCountsPath = userhome + "/delta/customer_counts/"

**Note: This lab depends upon the complete exectuion of the notebook titled "Open-Source-Delta-Lake" and the "Delta-Lake-Basics" lab. If these tables don't exist, go back and run all cells in these notebook.**

### Time Travel
Because Delta Lake is version controlled, you have the option to query past versions of the data. Let's look at the history of our current Delta table.

In [0]:
%sql
DESCRIBE HISTORY customer_data_delta

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
3,2021-06-25T00:05:30.000+0000,5614161306485076,vanitha.a.sivakumar@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(2423029523525614),0624-234608-pawl774,2,WriteSerializable,true,"Map(numFiles -> 43, numOutputBytes -> 1686992, numOutputRows -> 99999)",null
2,2021-06-24T18:32:18.000+0000,5614161306485076,vanitha.a.sivakumar@gmail.com,MERGE,"Map(predicate -> ((CAST(spark_catalog.default.customer_data_delta.`InvoiceNo` AS BIGINT) = upsert_data.`InvoiceNo`) AND (spark_catalog.default.customer_data_delta.`StockCode` = upsert_data.`StockCode`)), matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(2786064228246723),0624-174902-macho720,1,WriteSerializable,false,"Map(numTargetRowsCopied -> 61248, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 4, executionTimeMs -> 12416, numTargetRowsInserted -> 10, scanTimeMs -> 6173, numTargetRowsUpdated -> 4, numOutputRows -> 61262, numTargetChangeFilesAdded -> 0, numSourceRows -> 14, numTargetFilesRemoved -> 4, rewriteTimeMs -> 6158)",null
1,2021-06-24T18:13:15.000+0000,5614161306485076,vanitha.a.sivakumar@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [""country""])",null,List(2786064228246723),0624-174902-macho720,0,WriteSerializable,true,"Map(numFiles -> 2, numOutputBytes -> 5387, numOutputRows -> 36)",null
0,2021-06-24T17:52:02.000+0000,5614161306485076,vanitha.a.sivakumar@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [""Country""])",null,List(2786064228246723),0624-174902-macho720,null,WriteSerializable,false,"Map(numFiles -> 37, numOutputBytes -> 637539, numOutputRows -> 65499)",null


Querying an older version is as easy as adding `VERSION AS OF desired_version`. Let's verify that our table from one version back still exists.

In [0]:
%sql
SELECT COUNT(*)
FROM customer_data_delta
VERSION AS OF 1

count(1)
65535


Using a single file storage system, you now have access to every version of your historical data, ensuring that your data analysts will be able to replicate their reports (and compare aggregate changes over time) and your data scientists will be able to replicate their experiments.

### Check difference between versions

You want to compare how many orders from Sweden were added by your recent UPSERT to your BI table.

Let's start by getting the total sum of our `total_orders` column where our country is Sweden.

In [0]:
# TODO
count = spark.sql("SELECT SUM(total_orders) FROM customer_counts WHERE Country = 'Sweden'").collect()[0][0]

In [0]:
count

Out[25]: 2985

Again, we can look at the history of our Delta table here.

In [0]:
%sql
DESCRIBE HISTORY customer_counts

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
1,2021-06-25T11:39:36.000+0000,5614161306485076,vanitha.a.sivakumar@gmail.com,MERGE,"Map(predicate -> ((spark_catalog.vanitha_a_sivakumar_gmail_com_db.customer_counts.`Country` = spark_catalog.vanitha_a_sivakumar_gmail_com_db.new_customer_counts.`Country`) AND (spark_catalog.vanitha_a_sivakumar_gmail_com_db.customer_counts.`CustomerID` = spark_catalog.vanitha_a_sivakumar_gmail_com_db.new_customer_counts.`CustomerID`)), matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(2423029523525564),0625-113253-wreak337,0,WriteSerializable,false,"Map(numTargetRowsCopied -> 7, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 22, executionTimeMs -> 20166, numTargetRowsInserted -> 38124, scanTimeMs -> 7845, numTargetRowsUpdated -> 14, numOutputRows -> 38145, numTargetChangeFilesAdded -> 0, numSourceRows -> 38138, numTargetFilesRemoved -> 1, rewriteTimeMs -> 12238)",null
0,2021-06-25T11:39:00.000+0000,5614161306485076,vanitha.a.sivakumar@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [""Country""])",null,List(2423029523525564),0625-113253-wreak337,null,WriteSerializable,false,"Map(numFiles -> 25, numOutputBytes -> 27352, numOutputRows -> 1245)",null


Our original table will be version `0`. Let's write a SQL query to see how many orders we originally had from Sweden.

In [0]:
%sql
SELECT SUM(total_orders)
FROM customer_counts
VERSION AS OF 0
WHERE Country='Sweden'

sum(total_orders)
60


We can combine these two queries and get our difference, which represents our new entries.

In [0]:
%sql
SELECT SUM(total_orders) - (
  SELECT SUM(total_orders)
  FROM customer_counts
  VERSION AS OF 0
  WHERE Country='Sweden') AS new_entries
FROM customer_counts
WHERE Country='Sweden'

new_entries
2925


### OPTIMIZE and ZORDER

Let's apply some of these optimizations to `../delta/customer-data/`.

Our data is partitioned by `Country`.

We want to query the data for `StockCode` equal to `22301`.

We expect this query to be slow because we have to examine ALL OF `../delta/customer-data/` to find the desired `StockCode` and not just in one or two partitions.

First, let's time the above query: you will need to form a DataFrame to pass to `preZorderQuery`.

In [0]:
# TODO
%timeit preZorderQuery = spark.sql("SELECT * FROM delta.`{}` WHERE StockCode = 22301".format(DataPath) ).collect()

10.2 s ± 646 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Compact the files and re-order by `StockCode`.

In [0]:
%sql
-- TODO
OPTIMIZE customer_data_delta
ZORDER by StockCode

path,metrics
null,"List(26, 75, List(3390, 594364, 79919.0, 26, 2077918), List(2413, 471599, 29913.0, 75, 2243534), 33, List(minCubeSize(107374182400), List(0, 0), List(82, 2343550), 0, List(75, 2243534), 26, null), 1, 82, 7, false)"


Let's time the above query again: you will need to form a DataFrame to pass to `postZorderQuery`.

In [0]:
# TODO
%timeit postZorderQuery = spark.sql("SELECT * FROM delta.`{}` WHERE StockCode = 22301".format(DataPath) ).collect() 

3.8 s ± 220 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

### OPTIMIZE your BI table

Here we'll optimize our `customer_counts` table so that we can quickly query on our `CustomerID` column.

In [0]:
%sql
OPTIMIZE customer_counts
ZORDER by (CustomerID)

path,metrics
null,"List(13, 26, List(7018, 15537, 9247.0, 13, 120215), List(807, 9464, 4968.0, 26, 129171), 33, List(minCubeSize(107374182400), List(0, 0), List(46, 215322), 0, List(26, 129171), 13, null), 1, 46, 20, false)"


Now we can easily look at which of our customers have made the most orders.

In [0]:
%sql
SELECT CustomerID, SUM(total_orders) AS total
FROM customer_counts
GROUP BY CustomerID
ORDER BY total DESC

CustomerID,total
null,25281
12748,695
17841,481
14606,421
15311,418
14911,377
17850,297
13089,261
13081,261
18118,250


Or we can see examine those customers that operate in the most countries.

In [0]:
%sql
SELECT CustomerID, COUNT(Country) AS num_countries
FROM customer_counts
GROUP BY CustomerID
SORT BY num_countries DESC

CustomerID,num_countries
20123,22
20059,21
20453,21
20586,21
20600,21
20705,21
20807,21
21095,21
21137,21
21160,21


And then look at how many orders a customer made in each of these countries.

In [0]:
%sql
SELECT Country, total_orders
FROM customer_counts
WHERE CustomerID = 20059

Country,total_orders
United Kingdom,8
France,4
null,5
Germany,4
United States,7
Canada,2
Japan,3
Switzerland,2
Netherlands,2
Italy,2


### Using VACUUM to clean up small files

After we run OPTIMIZE, we have a number of uncompacted files that are no longer necessary. Running VACUUM will remove these files for us.

Let's go ahead and VACUUM our `customer_data_delta` table, which points at the files in our `DeltaPath` variable.

Count number of files before `VACUUM` for `Country=Sweden`.

In [0]:
# TODO
preNumFiles = len(dbutils.fs.ls(DeltaPath+"\Country=Sweden"))
print(preNumFiles)

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-2423029523525604> in <module> 
 1 # TODO 
 ----> 2 preNumFiles = len ( dbutils . fs . ls ( DeltaPath + "\Country=Sweden" ) ) 
 3 print ( preNumFiles ) 

 /databricks/python_shell/scripts/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 323 exc . __context__ = None 
 324 exc . __cause__ = None 
 --> 325 raise exc
 326 return f_with_exception_handling
 327 

 ExecutionError : An error occurred while calling z:com.databricks.backend.daemon.dbutils.FSUtils.ls.
: java.io.FileNotFoundException: /user/vanitha.a.sivakumar@gmail.com/delta/customer-data/\Country=Sweden
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:120)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:68)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV1.listStatus(DatabricksFileSystemV1.scala:177)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystem.listStatus(DatabricksFileSystem.scala:150)
	at com.databricks.backend.daemon.dbutils.FSUtils$.$anonfun$ls$1(DBUtilsCore.scala:153)
	at com.databricks.backend.daemon.dbutils.FSUtils$.withFsSafetyCheck(DBUtilsCore.scala:90)
	at com.databricks.backend.daemon.dbutils.FSUtils$.ls(DBUtilsCore.scala:152)
	at com.databricks.backend.daemon.dbutils.FSUtils.ls(DBUtilsCore.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

If you try to perform an immediate `VACUUM` (using `RETAIN 0 HOURS` to clean up recently optimized files), you will get an error.

In [0]:
%sql
-- TODO
VACUUM customer_data_delta RETAIN 0 HOURS;

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: java.lang.IllegalArgumentException: requirement failed: Are you sure you would like to vacuum files with such a low retention period? If you have
writers that are currently writing to this table, there is a risk that you may corrupt the
state of your Delta table.

If you are certain that there are no operations being performed on this table, such as
insert/upsert/delete/optimize, then you may turn off this check by setting:
spark.databricks.delta.retentionDurationCheck.enabled = false

If you are not sure, please use a value not less than "168 hours".
 
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.sql.transaction.tahoe.commands.VacuumCommand$.checkRetentionPeriodSafety(VacuumCommand.scala:81)
	at com.databricks.sql.transaction.tahoe.commands.VacuumCommand$.$anonfun$gc$2(VacuumCommand.scala:120)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:80)
	at com.databricks.sql.transaction.tahoe.metering.DeltaLogging.$anonfun$recordDeltaOperation$5(DeltaLogging.scala:115)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$4(UsageLogging.scala:434)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:240)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:235)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:232)
	at com.databricks.spark.util.PublicDBLogging.withAttributionContext(DatabricksSparkUsageLogger.scala:18)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:279)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:271)
	at com.databricks.spark.util.PublicDBLogging.withAttributionTags(DatabricksSparkUsageLogger.scala:18)
	at com.databricks.logging.UsageLogging.recordOperation(UsageLogging.scala:415)
	at com.databricks.logging.UsageLogging.recordOperation$(UsageLogging.scala:341)
	at com.databricks.spark.util.PublicDBLogging.recordOperation(DatabricksSparkUsageLogger.scala:18)
	at com.databricks.spark.util.PublicDBLogging.recordOperation0(DatabricksSparkUsageLogger.scala:55)
	at com.databricks.spark.util.DatabricksSparkUsageLogger.recordOperation(DatabricksSparkUsageLogger.scala:98)
	at com.databricks.spark.util.UsageLogger.recordOperation(UsageLogger.scala:70)
	at com.databricks.spark.util.UsageLogger.recordOperation$(UsageLogger.scala:57)
	at com.databricks.spark.util.DatabricksSparkUsageLogger.recordOperation(DatabricksSparkUsageLogger.scala:67)
	at com.databricks.spark.util.UsageLogging.recordOperation(UsageLogger.scala:374)
	at com.databricks.spark.util.UsageLogging.recordOperation$(UsageLogger.scala:353)
	at com.databricks.sql.transaction.tahoe.commands.VacuumCommand$.recordOperation(VacuumCommand.scala:54)
	at com.databricks.sql.transaction.tahoe.metering.DeltaLogging.recordDeltaOperation(DeltaLogging.scala:113)
	at com.databricks.sql.transaction.tahoe.metering.DeltaLogging.recordDeltaOperation$(DeltaLogging.scala:97)
	at com.databricks.sql.transaction.tahoe.commands.VacuumCommand$.recordDeltaOperation(VacuumCommand.scala:54)
	at com.databricks.sql.transaction.tahoe.commands.VacuumCommand$.$anonfun$gc$1(VacuumCommand.scala:106)
	at com.databricks.sql.acl.CheckPermissions$.trusted(CheckPermissions.scala:1212)
	at com.databricks.sql.transaction.tahoe.commands.VacuumCommand$.gc(VacuumCommand.scala:106)
	at com.databricks.sql.transaction.directory.VacuumTableCommand.$anonfun$run$1(VacuumTableCommand.scala:58)
	at com.databricks.sql.acl.CheckPermissions$.trusted(CheckPermissions.scala:1230)
	at com.databricks.sql.transaction.directory.VacuumTableCommand.run(VacuumTableCommand.scala:43)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:73)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(c

This is a helfpul error. Remember that `VACUUM` is intended for occasional garbage collection. Here we'll just demonstrating that we _can_ use it to clean up files, so we'll set our configuration to allow this operation.

In [0]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", False)

Now we won't get an error when we run `VACUUM`.

In [0]:
%sql
VACUUM customer_data_delta RETAIN 0 HOURS;

path
dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data


Count how many files there are for `Country=Sweden`.

In [0]:
# TODO
postNumFiles = len(dbutils.fs.ls(DeltaPath + "/Country=Sweden"))
print(postNumFiles)

1

Comparing our `preNumFiles` to `postNumFiles`, we can see that this number has reduced.

In [0]:
dbutils.fs.ls(DeltaPath)

Out[41]: [FileInfo(path='dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data/Country=Australia/', name='Country=Australia/', size=0),
 FileInfo(path='dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data/Country=Austria/', name='Country=Austria/', size=0),
 FileInfo(path='dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data/Country=Bahrain/', name='Country=Bahrain/', size=0),
 FileInfo(path='dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data/Country=Belgium/', name='Country=Belgium/', size=0),
 FileInfo(path='dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data/Country=Brazil/', name='Country=Brazil/', size=0),
 FileInfo(path='dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data/Country=Canada/', name='Country=Canada/', size=0),
 FileInfo(path='dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data/Country=Channel Islands/', name='Country=Channel Islands/', size=0),
 FileInfo(path='dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data/Country=China/', name='Country=China/', size=0),
 FileInfo(path='dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data/Country=Cyprus/', name='Country=Cyprus/', size=0),
 FileInfo(path='dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data/Country=Denmark/', name='Country=Denmark/', size=0),
 FileInfo(path='dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data/Country=EIRE/', name='Country=EIRE/', size=0),
 FileInfo(path='dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data/Country=Finland/', name='Country=Finland/', size=0),
 FileInfo(path='dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data/Country=France/', name='Country=France/', size=0),
 FileInfo(path='dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data/Country=Germany/', name='Country=Germany/', size=0),
 FileInfo(path='dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data/Country=Iceland/', name='Country=Iceland/', size=0),
 FileInfo(path='dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data/Country=Israel/', name='Country=Israel/', size=0),
 FileInfo(path='dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data/Country=Italy/', name='Country=Italy/', size=0),
 FileInfo(path='dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data/Country=Japan/', name='Country=Japan/', size=0),
 FileInfo(path='dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data/Country=Korea/', name='Country=Korea/', size=0),
 FileInfo(path='dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data/Country=Lithuania/', name='Country=Lithuania/', size=0),
 FileInfo(path='dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data/Country=Mexico/', name='Country=Mexico/', size=0),
 FileInfo(path='dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data/Country=Netherlands/', name='Country=Netherlands/', size=0),
 FileInfo(path='dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data/Country=Norway/', name='Country=Norway/', size=0),
 FileInfo(path='dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data/Country=Poland/', name='Country=Poland/', size=0),
 FileInfo(path='dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data/Country=Portugal/', name='Country=Portugal/', size=0),
 FileInfo(path='dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data/Country=Sierra Leone/', name='Country=Sierra Leone/', size=0),
 FileInfo(path='dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data/Country=Singapore/', name='Country=Singapore/', size=0),
 FileInfo(path='dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data/Country=Spain/', name='Country=Spain/', size=0),
 FileInfo(path='dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data/Country=Sweden/', name='Country=Sweden/', size=0),
 FileInfo(path='dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data/Country=Switzerland/', name='Country=Switzerland/', size=0),
 FileInfo(path='dbfs:/user/vanitha.a.sivakumar@gmail.com/delta/customer-data/Co